## Library Imports

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix

import itertools
import os
import shutil
import random
from time import time
import matplotlib.pyplot as plt
#import pillow as pil
%matplotlib inline

In [54]:
DATA_PATH = 'SampleData'
#NUM_LAYERS_TO_REMOVE = 5
#NUM_LAYERS_TO_TRAIN = 20
LAYERS_TO_UNFREEZE = 0

## Create image pre-processing function

In [4]:
# Function to preprocess the image with Keras
preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input
#def image_preprocessing(file):
#    img = image.load_img(file, target_size=(224, 224))
#    img_array = image.img_to_array(img)
#    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
#    return tf.keras.applications.mobilenet_v3.preprocess_input(img_array_expanded_dims)

## Load the metadata file.  Create a ground truth column.

In [5]:
# Load the metadata file
#metadata = pd.read_excel('metadata_modified.xlsx')
#image_path = metadata['Image Path']
# Create the ground truth values and put it into a new 'g_truth' column
#g_truth = metadata['Malignant'] + metadata['A']*2 + metadata['F']*4 + metadata['PT']*8 + metadata['TA']*16 + metadata['DC']*32 + metadata['LC']*64 + metadata['MC']*128 + metadata['PC']*256
#metadata['g_truth'] = g_truth

## Display a test image

In [6]:
# Display test image
#from IPython.display import Image
#Image(filename=image_path[0], width=300,height=200)

In [7]:
mobile_v3 = tf.keras.applications.MobileNetV3Large()
#preprocessed_image = image_preprocessing(image_path[7117])
#predictions = mobile_v3.predict(preprocessed_image)
#print("Output shape: ", predictions.shape)
#results = imagenet_utils.decode_predictions(predictions)
#results

Remove the specifiied number of layers.
Freeze the specified number of layers.

In [8]:
# Remove last layers
#x = mobile_v3.layers[-NUM_LAYERS_TO_REMOVE].output
#output = Dense(units=2, activation='softmax')(x)

# Freeze layers
#for layer in new_model.layers[:-NUM_LAYERS_TO_TRAIN]:
#    layer.trainable = False                            

## Define the training and validation datasets

In [9]:
path_to_files = 'BreaKHis_v1/BreaKHis_v1/histology_slides/breast/'
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  path_to_files,
  validation_split=0.25,
  subset="training",
  seed=42,
  image_size=(224, 224),
#  batch_size=32,
  shuffle=True
)
  
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  path_to_files,
  validation_split=0.25,
  subset="validation",
  seed=42,
  image_size=(224, 224),
#  batch_size=32,
  shuffle=True
)
#val_batches = tf.data.experimental.cardinality(val_dataset)
#val_dataset = val_dataset.take((2*val_batches) // 5)
#test_dataset = val_dataset.skip((2*val_batches) // 5)


Found 7909 files belonging to 8 classes.
Using 5932 files for training.
Found 7909 files belonging to 8 classes.
Using 1977 files for validation.


## Use AUTOTUNE to decrease I/O roadblocks

In [10]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
#test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

## Define the model

In [11]:
IMG_SIZE=(224,224)
IMG_SHAPE = IMG_SIZE + (3,)
def new_model (image_shape=IMG_SIZE):
    input_shape = image_shape + (3,)
    base_model = tf.keras.applications.MobileNetV3Large(input_shape=IMG_SHAPE,
                                                        include_top=False, # important
                                                        weights='imagenet')
    # freeze the base model
    base_model.trainable = False
    # create input layer
    inputs = tf.keras.Input(shape=input_shape)
    # pre-process inputs
    x = preprocess_input(inputs)
    # set training to False to avoid tracking statistics in batch norm layer
    x = base_model(x, training=False)
    # add new binary classification layers
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # include dropout with probability of 0.2 to avoid overfitting
    x = tf.keras.layers.Dropout(0.2)(x)

    # user prediction layer of one neuron (binary classifier only needs one - multiclass may need more)
    prediction_layer = tf.keras.layers.Dense(1)
    outputs = prediction_layer(x)

    model = tf.keras.Model(inputs,outputs)
    return model

In [55]:
IMG_SIZE=(224,224)
IMG_SHAPE = IMG_SIZE + (3,)
def new_model2 (image_shape=IMG_SIZE):
    input_shape = image_shape + (3,)
    base_model = tf.keras.applications.MobileNetV3Large(input_shape=IMG_SHAPE,
                                                        include_top=False, # important
                                                        weights='imagenet')
    
    # freeze the base model
    base_model.trainable = False

    # unfreeze some layers
    for layer in base_model.layers[-LAYERS_TO_UNFREEZE:]:
        layer.trainable = True

    print("Number of layers in the base model: ", len(base_model.layers))

    # create input layer
    inputs = tf.keras.Input(shape=input_shape)
    # pre-process inputs
    x = preprocess_input(inputs)
    # set training to False to avoid tracking statistics in batch norm layer
    x = base_model(x, training=False)
    # add flatten layer
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(256, activation='relu')(x)
    #x = tf.keras.layers.Dense(128, activation='relu')(x)
    #x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(8, activation='relu')(x)

    # include dropout with probability of 0.2 to avoid overfitting
    #x = tf.keras.layers.Dropout(0.2)(x)

    # user prediction layer of one neuron (binary classifier only needs one - multiclass may need more)
#    prediction_layer = tf.keras.layers.Dense(1)
#    outputs = prediction_layer(x)
    outputs = x

    model = tf.keras.Model(inputs,outputs)
    return model

In [58]:
# Define the learning rate scheduler
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers.schedules import ExponentialDecay

def lr_scheduler(epoch, lr, decay_rate=0.8, decay_step=1):
    if epoch % decay_step == 0:
        return lr * decay_rate
    return lr

def lr_scheduler2(lr, epoch_decay, layer_decay, model):
    
    trainable_layers = []
    learning_rate_schedule = {}
    # Get all the trainable layers
    for layer in model.layers:
        if layer.name.startswith('Mobilenet'):
            for mobilenet_layer in layer.layers:
                if mobilenet_layer.trainable:
                    pass
                    #trainable_layers.append(mobilenet_layer.name)
        elif layer.trainable:
            trainable_layers.append(layer.name)

    # Create the learning rate schedule
    for layer in trainable_layers:
        learning_rate_schedule[layer] = ExponentialDecay(initial_learning_rate=lr, decay_steps=1, decay_rate=epoch_decay)

    print(learning_rate_schedule)

    return learning_rate_schedule

    #print('Trainable layers: ', len(trainable_layers))
    #print('Trainable layers: ', trainable_layers)

## Train the model

In [64]:
project5_model = new_model2(IMG_SIZE)

print(project5_model.summary())

lr_schedule = lr_scheduler2(0.001, 0.5, 0.5, project5_model)



#base_learning_rate = 0.001
#opt = tf.keras.optimizers.legacy.SGD(learning_rate=base_learning_rate, momentum=0.9)
#project5_model.compile(optimizer=opt,
#            loss=tf.keras.losses.MeanAbsoluteError(),
#            metrics=["accuracy"])
adam_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule)
#callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler(decay_rate=0.5))
project5_model.compile(optimizer=adam_optimizer,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

initial_epochs = 10
history = project5_model.fit(
    train_dataset, 
    epochs=initial_epochs,
    #callbacks=[callback],
    verbose=1,
    validation_data=val_dataset)
project5_model.summary()

Number of layers in the base model:  263
Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_57 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 flatten_28 (Flatten)        (None, 47040)             0         
                                                                 
 dense_60 (Dense)            (None, 32)                1505312   
                                                                 
 dense_61 (Dense)            (None, 8)                 264       
                                                                 
Total params: 4501928 (17.17 MB)
Trainable params: 1505576 (5.74 MB)
Non-trainable 

ValueError: in user code:

    File "/Users/kdevoe/Documents/CS/Masters/AAI501/Group Project/aai501-su23-group-1/env6/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/kdevoe/Documents/CS/Masters/AAI501/Group Project/aai501-su23-group-1/env6/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/kdevoe/Documents/CS/Masters/AAI501/Group Project/aai501-su23-group-1/env6/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/kdevoe/Documents/CS/Masters/AAI501/Group Project/aai501-su23-group-1/env6/lib/python3.10/site-packages/keras/src/engine/training.py", line 1084, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/Users/kde

In [ ]:
loss0, accuracy0 = project5_model.evaluate(val_dataset)

print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))